In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

nbPicture = 0

import os
for dirname, _, filenames in os.walk('/kaggle/input/radiolarian-images/Radiolarian images'):
    for filename in filenames:
        nbPicture += 1
print (nbPicture)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1087


We have checked the number of picture avaible, now let's go into the structure.

We use the model that already exists in the Tensorflow and we just add a Dropout layer and a 8-dimensional Dense layer, to know which one of the 8 classes we have in our data is likely to be the class of the input.

We then just compile the model with the right metrics and accuracy regarding the representation of the classes we have chosen (a vectorial one).

In [2]:
# Based on the work done in the notebook "Test MobileNet"
import gc
import tensorflow as tf
import numpy as np

from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model



target_size = 120

def build_model( ):
    input_tensor = Input(shape=(target_size, target_size, 3))
    base_model = MobileNet(
        dropout = 0.15,
        include_top=False,
        weights='imagenet',
        input_tensor=input_tensor,
        input_shape=(target_size, target_size, 3),
        pooling='avg')
    
    for layer in base_model.layers:
        layer.trainable = True  # trainable has to be false in order to freeze the layers
            
    #op = Dense(256, activation='relu')(base_model.output)
    op = Dropout(.5)(base_model.output)
        
        ##
        # softmax: calculates a probability for every possible class.
        #
        # activation='softmax': return the highest probability;
        # for example, if 'Coat' is the highest probability then the result would be 
        # something like [0,0,0,0,1,0,0,0,0,0] with 1 in index 5 indicate 'Coat' in our case.
        ##
    output_tensor = Dense(8, activation='softmax')(op)
    
    model = Model(inputs=input_tensor, outputs=output_tensor)
    
    
    return model
    
    
from tensorflow.keras.optimizers import Adam


model = build_model()
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


17227776/17225924 [==============================] - 1s 0us/step


Here, we load the data from the Kaggle dataset that I've created. Then, we split the data into a train set and a test set (85/15).

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

batchSize = 64

# Load the train data and test data

train_data_dir = '/kaggle/input/radiolarian-images/Radiolarian images'


train_datagen = ImageDataGenerator()
itr = train_datagen.flow_from_directory(
train_data_dir,
target_size=(target_size, target_size),
batch_size=nbPicture,
class_mode='categorical')

X, y = itr.next()

print("X shape = ", X.shape)
print("y shape = ", y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
    
print("x_train shape = ", x_train.shape)
print("x_test shape = ", x_test.shape)
print("y_train shape = ", y_train.shape)
print("y_test shape = ", y_test.shape)
    

Found 1087 images belonging to 8 classes.
X shape =  (1087, 120, 120, 3)
y shape =  (1087, 8)
x_train shape =  (923, 120, 120, 3)
x_test shape =  (164, 120, 120, 3)
y_train shape =  (923, 8)
y_test shape =  (164, 8)


The preprocess_images is the function taking as input a dataset and return the same dataset, but fitting the prerequisites of the model we have created. We take care of the format of our data, checking wether it has 3 or 4 dimensions, and if the images are in grayscale or in a rgb format.

In [4]:
from sklearn.utils import shuffle
from tensorflow.image import resize, grayscale_to_rgb
    
    
def preprocess_images(x):

        
    

    print(x.shape)
    batch = preprocess_input(x)
    if (len(x.shape)==3):
        
      
        
        batch = batch[..., np.newaxis]
        batch = resize(batch, (target_size, target_size))

    if (x.shape[3]==1):
        batch = grayscale_to_rgb(tf.convert_to_tensor(batch))
    batch = np.asarray(batch)
   
           
    return (tf.convert_to_tensor(batch.astype(np.float32)))
    

We apply the function on our train dataset, and then train the model.
As the dataset isn't big enough, I make it bigger artificially by repeating it.

In [5]:
norm_x_train = preprocess_images(x_train)

del x_train
gc.collect()
    
train_dataset = tf.data.Dataset.from_tensor_slices((norm_x_train, y_train)).shuffle(norm_x_train.shape[0], reshuffle_each_iteration=True).batch(batchSize).repeat(70)
    
    
del norm_x_train
del y_train
gc.collect()


    
model.fit(
        train_dataset,
        batch_size = batchSize,
        steps_per_epoch=100,
        verbose=1,
        epochs=5)
    

del train_dataset
gc.collect()

(923, 120, 120, 3)
Epoch 1/5
100/100 [==============================] - 16s 83ms/step - loss: 0.8588 - categorical_accuracy: 0.7834
Epoch 2/5
100/100 [==============================] - 8s 80ms/step - loss: 0.0843 - categorical_accuracy: 0.9758
Epoch 3/5
100/100 [==============================] - 8s 80ms/step - loss: 0.1053 - categorical_accuracy: 0.9741
Epoch 4/5
100/100 [==============================] - 8s 80ms/step - loss: 0.0393 - categorical_accuracy: 0.9848
Epoch 5/5
100/100 [==============================] - 8s 79ms/step - loss: 0.0115 - categorical_accuracy: 0.9965


8

We apply the function on our test dataset, and then train the model.
We do the same thing about repeating the dataset as it isn,'t big enough.

In [6]:
norm_x_test = preprocess_images(x_test)

test_dataset = tf.data.Dataset.from_tensor_slices((norm_x_test, y_test)).shuffle(norm_x_test.shape[0], reshuffle_each_iteration=True).batch(batchSize).repeat(10)

del norm_x_test
del y_test
gc.collect()
    
    
model.evaluate(
        test_dataset,
        steps=10,
        verbose=1)

(164, 120, 120, 3)
10/10 [==============================] - 1s 27ms/step - loss: 0.2789 - categorical_accuracy: 0.9388


[0.27891838550567627, 0.9388489127159119]